In [1]:
import mlflow
from dotenv import load_dotenv
from databricks.connect import DatabricksSession

spark = DatabricksSession.builder.getOrCreate()
load_dotenv('../.env')

DATABRICKS_HOST = os.getenv('host')
DATABRICKS_TOKEN = os.getenv('token')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
print(f"host: {DATABRICKS_HOST}")

mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_registry_uri("http://localhost:5000")
mlflow.set_experiment("crewai_flow_local_dev")
mlflow.crewai.autolog()

host: adb-984752964297111.11.azuredatabricks.net


2025/03/05 14:54:42 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/q.yu/workspace/agents_use_cases/.venv/lib/python3.11/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed"


In [2]:
from unitycatalog.ai.core.base import set_uc_function_client
from unitycatalog.ai.core.databricks import DatabricksFunctionClient

client = DatabricksFunctionClient()
set_uc_function_client(client)

CATALOG = 'fins_genai'
SCHEMA = 'agents'

In [3]:
df = spark.sql(f"select distinct(intent) from {CATALOG}.{SCHEMA}.call_transcripts")
display(df.show())

+-----------------+
|           intent|
+-----------------+
|motocycle related|
|    policy change|
|     car accident|
|    home accident|
|            theft|
| general question|
+-----------------+



None

In [4]:
uc_tools = [f"{CATALOG}.{SCHEMA}.{func.name}" for func in client.list_functions(catalog=CATALOG,
                                                                                schema=SCHEMA)
            if func.name.endswith('phone_number') or func.name == 'search_policy']
print(uc_tools)

['fins_genai.agents.get_customer_profile_by_phone_number', 'fins_genai.agents.get_customer_transcripts_by_phone_number', 'fins_genai.agents.search_policy']


# Create a Crew

## Set tools

In [5]:
from unitycatalog.ai.crewai.toolkit import UCFunctionToolkit

toolkit = UCFunctionToolkit(function_names=uc_tools)
tools = toolkit.tools

cs_profile_tool, find_transcript_tool, search_policy_tool = tools

In [6]:
from crewai import Agent, Task, Crew

# Intent detect crew
# This can be a an user inputs
intent_analyzer = Agent(
    role="Transcript Analyzer",
    goal="Analyze the customer transcripts and detect the customer intent based on his or her phone number: {phone_number}",
    backstory="The agent is responsible for analyzing the customer transcripts and identifying the customer intent.",
    tools=[find_transcript_tool],
    verbose=True
)

intent_analysis = Task(
    description="Given a customer based on his or her phone number: {phone_number} analyze the call transcripts"
                "carefully and detect the customer intent.",
    expected_output="customer intent in a few words",
    agent=intent_analyzer
)

intend_detect_crew = Crew(
    agents=[intent_analyzer],
    tasks=[intent_analysis],
    verbose=True,
    planning=True
)

claim_processor = Agent(
    role="Insurance Claim Process",
    goal="Process the insurance claim by customer with phone number: {phone_number} based on the "
         "customer's profile and call transcript information.",
    backstory="The agent is responsible for processing the insurance claim based "
              "on the customer's profile and call transcript information.",
    tools=[cs_profile_tool, find_transcript_tool],
    verbose=True
)

file_claim = Task(
    description="Pull customer profile based on his or her phone number: {phone_number}. Use call"
                " transcript as context to process the insurance claim.",
    expected_output="A processed insurance claim based on the customer's profile and call "
                    "transcript information.",
    agent=claim_processor,
)

claim_processor_crew = Crew(
    agents=[claim_processor],
    tasks=[file_claim],
    verbose=True,
    planning=True
)

policy_helper = Agent(
    role="Insurance Policy Helper",
    goal="Help to answer customer questions on policy or perform policy change tasks using policy "
         "document as reference. Also refer to customer profile and transcripts",
    backstory="The agent is responsible for performing policy change tasks using policy",
    tools=[cs_profile_tool, find_transcript_tool, search_policy_tool],
    verbose=True
)

search_retrieve_policy = Task(
    description="Search for the policy based on the customer with phone number: {phone_number} 's"
                " profile and transcript information.",
    expected_output="A detailed and polite response that addresses customer's inquery on policy.",
    agent=policy_helper
)

policy_answer_crew = Crew(
    agents=[policy_helper],
    tasks=[search_retrieve_policy],
    verbose=True,
    planning=True
)


In [23]:
import asyncio
from typing import List

from crewai.flow.flow import Flow, listen, router, start
from pydantic import BaseModel
import re

class InsuranceOperatorFlow(Flow):

    @start()
    def intend_triage(self):
        if self.state['user_input']:
            pattern = r"\(\d{3}\)-\d{3}-\d{4}"
            match = re.search(pattern, self.state['user_input'])
            print(match)
            if match:
                self.state['intent'] = intend_detect_crew.kickoff({'phone_number': match.group()})
            else:
                self.state['intent'] = "Cannot identify customer"

    @router(intend_triage)
    def assign_task_to_crew(self):
        if "policy" in self.state['intent']:
            return "policy"
        elif any(keyword in self.state['intent']
                 for keyword in ["accident", "theft", "claim", "motor"]):
            return "claim"
        else:
            return "Cannot identify customer intent, terminate"

    @listen("claim")
    def file_a_claim(self):
        claim_processor_crew.kickoff({'phone_number': self.state['user_input']})

    @listen("policy")
    def policy_question(self):
        policy_answer_crew.kickoff({'phone_number': self.state['user_input']})

In [24]:
flow = InsuranceOperatorFlow()
flow.plot()

Plot saved as crewai_flow.html


In [25]:
from IPython.display import IFrame

IFrame(src='./crewai_flow.html', width='100%', height=800)

In [26]:
result = await flow.kickoff_async(inputs={'user_input': 'my number is (875)-209-8555'})
result

<re.Match object; span=(13, 27), match='(875)-209-8555'>


2025/03/05 15:01:24 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/q.yu/workspace/agents_use_cases/.venv/lib/python3.11/site-packages/pydantic/functional_validators.py:787: UserWarning: Pydantic serializer warnings:
  Expected `InstanceOf` but got `CrewAgentExecutor` with value `<crewai.agents.crew_agent...r object at 0x33c2b5110>` - serialized value may not be as expected"


 
[2025-03-05 15:01:24][INFO]: Planning the crew execution
# Agent: Transcript Analyzer
## Task: Given a customer based on his or her phone number: (875)-209-8555 analyze the call transcriptscarefully and detect the customer intent.1. Start by identifying the customer phone number: (875)-209-8555. 
2. Utilize the available tool 'fins_genai__agents__get_customer_transcripts_by_phone_number' to retrieve the call transcript for the customer with that specific phone number. 
   2.1. Provide the input in the correct format, ensuring that the phone number is used as the 'search_name' argument in the tool. 
3. Execute the tool successfully and wait for the tool to return the customer transcript associated with the phone number. 
4. Once the transcript is retrieved, conduct a detailed analysis of the conversation by carefully reading through the entire text. 
5. Focus on identifying keywords, phrases, and overall context that indicate the purpose of the customer's call. 
6. Look for signals th

/Users/q.yu/workspace/agents_use_cases/.venv/lib/python3.11/site-packages/unitycatalog/ai/core/databricks.py:680: UserWarning: The following parameters do not have descriptions: search_name for the function fins_genai.agents.get_customer_transcripts_by_phone_number. Using Unity Catalog functions that do not have parameter descriptions limits the functionality for an LLM to understand how to call your function. To improve tool calling accuracy, provide verbose parameter descriptions that fully explain what the expected usage of the function arguments are.
  check_function_info(function_info)




# Agent: Transcript Analyzer
## Thought: I need to retrieve the customer transcript for the phone number (875)-209-8555.
## Using tool: fins_genai__agents__get_customer_transcripts_by_phone_number
## Tool Input: 
"{\"search_name\": \"(875)-209-8555\"}"
## Tool Output: 
{"format": "CSV", "value": "profile\n\"I need to file a claim for fire damage in my kitchen from a cooking accident.,   Sorry to hear that, but let's get one thing straight: we don't pay for foolish mistakes. If you're dripped grease on the stove and set the kitchen on fire, that's not our problem. You're gonna need to prove this was some kind of freak accident before we pay up. What's the point of even having insurance if you're just gonna be reckless in the kitchen? Alright, let's get the details. What's the make and model of your vehicle? And what's your policy number?,    I understand your concerns, but I assure you, this was not a foolish mistake. I am a responsible homeowner and I take great care in my cooking. T

'Cannot identify customer intent, terminate'

Trace(request_id=5e665487d4e74216a0305d63a8f9ad98)